In [1]:
import pandas as pd
import requests
import json
import glob
import os

## GET AUTHORISATION TOKEN
Grab your token and your client ID used to request an authorisation token (https://supercoach.heraldsun.com.au/2020/api/afl/classic/v1/access_token) and input them into the payload

In [2]:
# Start the session
session = requests.Session()

# Create the payload
payload = {
    "token": "eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6Ik1UVkdNRGxETUVaRk1qWkdOVU0yT0RnNFF6VTROalZCTlRsR1FrUTRSamM1TVROQ09UWTJOZyJ9.eyJodHRwOi8vbG9naW4ubmV3c2NvcnBhdXN0cmFsaWEuY29tLmF1L3Byb2ZpbGUiOnsidGhpbmtfaWQiOiIxNDEyMzIzMSIsImN1c3RvbWVyX3Byb2R1Y3RfaG9sZGluZyI6WyJaWl9OTF9SRUciLCJaWl9TQ19EREFGTCIsIlpaX1NDX0RERlJOT0VOVCJdLCJzaXRlIjoiSGVyYWxkU3VuIiwiaGFzX2NvbnNlbnRlZF90b190YyI6dHJ1ZSwiYXV0aFByb3ZpZGVyIjoiYXV0aDAifSwibmlja25hbWUiOiJNYXJrIiwibmFtZSI6Ik1hcmsiLCJwaWN0dXJlIjoiaHR0cHM6Ly9ncmFwaC5mYWNlYm9vay5jb20vNzE1NjE3NzU2L3BpY3R1cmUiLCJ1cGRhdGVkX2F0IjoiMjAyMC0wNy0xMVQwNzo1Mjo0Mi40NjBaIiwiaXNzIjoiaHR0cHM6Ly9sb2dpbi5uZXdzY29ycGF1c3RyYWxpYS5jb20vIiwic3ViIjoiYXV0aDB8YzVjZWVjOWItYWQ4NC00NWE5LWFlNWMtOTBiNjc1YzRiMWMxIiwiYXVkIjoiWllDb3RsaWhxYUd1YXFTc1N2dTBMMnZ4RGRRWEN3MTYiLCJpYXQiOjE1OTQ0NTM5NjIsImV4cCI6MTYwMTc5Nzk2MiwiYXRfaGFzaCI6InRESkRxZUJnRWpBa2N5dFNKUjdCZkEiLCJub25jZSI6IkFmTy5YSn5ZfklOaHh5M2dUZ0hyTC50QUdoRmt6ZWJjIn0.P01TNTizPXHobT3Yt8Md3gTUNkK7I9opZMilZ_iAHsU6-YHwuRQo0jjIdyqtsTvxhBzGpCsEeEez7h2C1kOnrsIF57m3O6DRnRoEeNyB52vP--z7A-zx1KrY2EuawqAVeWc5-mQCcYohSWmx75yhi0bQD3PoPbP6jNxUnmYT3O11_AbnZAM5UkB6jaAcYpsfUVXl6cB_z_R1iuWJvt5fSHJg53LPtL4waV61YEaVCbl8h9ON1Cz13-vsd_2dndxwC_CqA9gdLEw2HUF195ebtDMvVN6raYojvRZGg1EI9HVcCK0sP4q6VUspGDMZm2L97UaKCw7SYwA5zQ6WJ46bJw",
    "client_id": "Lf7m0371XCbMBlQ0fAFoGRJlfCs2JZpYvLU1uEvd",
    "client_secret": "",
    "grant_type": "social",
    "service": "auth0",
}


# Post the payload to the site to log in
s = session.post("https://supercoach.heraldsun.com.au/2020/api/afl/classic/v1/access_token", data=payload)
display(s.text)
access_token = json.loads(s.text)['access_token']

headers = {
    "Authorization": "Bearer " + access_token
}

'{"token_type":"Bearer","expires_in":604800,"access_token":"0f5b0b2ddc076dc1b909fc914bc80a618b1c592f","refresh_token":"a90b3d96fb026becc9cca0da9d4fd799388ee038"}'

## PLAYER LIST

In [3]:
def get_position_from_json(positions):
    positions_string = ''
    for position in positions:
        if len(positions_string) > 0:
            positions_string = positions_string + ' ' + position['position']
        else:
            positions_string = positions_string + position['position']
    return positions_string

In [4]:
s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/players-cf?embed=positions", headers=headers)
player_json = json.loads(s.text)
df_player_list = pd.DataFrame()

for player in player_json:
    positions_string = get_position_from_json(player['positions'])
    df_line = pd.DataFrame(
        {
            'player_id': [player['id']],
            'feed_id': [player['feed_id']],
            'first_name': [player['first_name']],
            'last_name': [player['last_name']],
            'team': [player['team']['abbrev']],
            'position': [pos for pos in [player['positions']]],
            'position': positions_string
        }
    )
    df_player_list = pd.concat([df_player_list, df_line])
df_player_list

,player_id,feed_id,first_name,last_name,team,position
0,151,293957,Brodie,Grundy,COL,RUC
0,469,290528,Max,Gawn,MEL,RUC
0,755,295467,Jack,Macrae,WBD,MID
0,76,293535,Lachie,Neale,BRL,MID
0,244,291570,Nat,Fyfe,FRE,MID
...,...,...,...,...,...,...
0,818,1016214,Ayce,Taylor,ADE,DEF
0,821,1016097,Henry,Crauford,ESS,RUC
0,822,1020224,Tom,Hird,ESS,MID
0,825,297668,Nicholas,Reid,WCE,FWD


## LADDER AND FIXTURE DATA

In [5]:
# specify round
round_number = 12
match_json_list = []

for rd in range(1, round_number + 1):
    s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/leagues/96/ladderAndFixtures?round={}&scores=true".format(rd), headers=headers)  
    match_json = json.loads(s.text)
    match_json_list.append(match_json)

## COACH IDs AND NAMES

In [6]:
df_coaches = pd.DataFrame()
first_match_json = match_json_list[0]

for coach in first_match_json['ladder']:
    df_line = pd.DataFrame(
        {
            'coach_id': [coach['userTeam']['user']['id']],
            'coach_team_id': [coach['userTeam']['id']],
            'coach_first_name': [coach['userTeam']['user']['first_name']],
            'coach_team_name': [coach['userTeam']['teamname']],
        }
    )
    df_coaches = pd.concat([df_coaches, df_line])
    
df_coaches

,coach_id,coach_team_id,coach_first_name,coach_team_name
0,86078,9806,Lester,HoweIMetYourMother
0,180650,33301,Jordan,"What's Up, Doch?"
0,45168,5524,Luke,The Dragon's Laird
0,300791,182,Paul,Coronaviney
0,21562,186,Mark,GetRichOrDieTryin
0,10857,189,Anthony,The Full Bonty
0,71052,9867,James,Bad Boys For Fyfe
0,8258,9856,Simon,It's always Sonny


## PLAYER MATCH RESULTS

Function to get player scores for each match

In [7]:
def get_player_score_data(scores, on_field_status, coach_id, coach_team_id, coach_first_name, coach_team_name):
    df_player_results = pd.DataFrame()
#     if on_field_status == True:
#         scoring_tag = 'scoring'
#     else:
#         scoring_tag = 'nonscoring'
    df_player_line = pd.DataFrame()
    for player in scores:
        player_id = player['player_id'],
        if on_field_status == True:
            points = player['points']
        else:
            points = player['player']['player_stats'][0]['points']
        played_position = player['position']
        df_player_line = pd.DataFrame(
            {
                'round': rd_number,
                'coach_id': coach_id,
                'coach_team_id': coach_team_id,
                'coach_first_name': coach_first_name,
                'coach_team_name': coach_team_name,
                'player_id': player_id,
                'played_position': played_position,
                'points': points,
                'on_field': on_field_status
            })
        df_player_results = pd.concat([df_player_results, df_player_line])
    return df_player_results

In [8]:
df_player_match_results = pd.DataFrame()

for match in match_json_list:
    for coach in match['ladder']:
        rd_number = coach['round']
        coach_id = coach['userTeam']['user']['id'],
        coach_team_id = coach['userTeam']['id'],
        coach_first_name = coach['userTeam']['user']['first_name'],
        coach_team_name = coach['userTeam']['teamname'],
        df_on_field_players = get_player_score_data(coach['userTeam']['scores']['scoring'], True, coach_id, coach_team_id, coach_first_name, coach_team_name)
        df_off_field_players = get_player_score_data(coach['userTeam']['scores']['nonscoring'], False, coach_id, coach_team_id, coach_first_name, coach_team_name)
        df_all_players = pd.concat([df_on_field_players, df_off_field_players])
        df_player_match_results = pd.concat([df_player_match_results, df_all_players])

df_player_match_results = df_player_match_results.merge(df_player_list, on='player_id')
df_player_match_results['round_number_alt'] = "R" + df_player_match_results['round'].astype('str')
df_player_match_results

,round,coach_id,coach_team_id,coach_first_name,coach_team_name,player_id,played_position,points,on_field,feed_id,first_name,last_name,team,position,round_number_alt
0,1,86078,9806,Lester,HoweIMetYourMother,153,DEF,156,True,291313,Jeremy,Howe,COL,DEF,R1
1,2,86078,9806,Lester,HoweIMetYourMother,153,DEF,126,True,291313,Jeremy,Howe,COL,DEF,R2
2,3,86078,9806,Lester,HoweIMetYourMother,153,DEF,88,True,291313,Jeremy,Howe,COL,DEF,R3
3,4,86078,9806,Lester,HoweIMetYourMother,153,DEF,112,True,291313,Jeremy,Howe,COL,DEF,R4
4,1,86078,9806,Lester,HoweIMetYourMother,789,DEF,152,True,240283,Shannon,Hurn,WCE,DEF,R1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2107,11,71052,9867,James,Bad Boys For Fyfe,71,FWD,24,True,293581,Lincoln,McCarthy,BRL,FWD,R11
2108,12,8258,9856,Simon,It's always Sonny,614,DEF,79,False,280819,Dylan,Grimes,RIC,DEF,R12
2109,12,45168,5524,Luke,The Dragon's Laird,493,FWD,84,True,992644,Braydon,Preuss,MEL,RUC FWD,R12
2110,12,300791,182,Paul,Coronaviney,795,FWD,96,True,240406,Josh J.,Kennedy,WCE,FWD,R12


In [9]:
df_player_match_results.to_csv('outputs/player_match_results.csv')

## LADDER

In [10]:
df_player_match_results = pd.DataFrame()

df_ladder = pd.DataFrame()

for match in match_json_list:
    for coach in match['ladder']:
        df_line = pd.DataFrame(
            {
                'round': coach['round'],
                'coach_id': [coach['userTeam']['user']['id']],
                'coach_team_id': [coach['userTeam']['id']],
                'coach_first_name': [coach['userTeam']['user']['first_name']],
                'coach_team_name': [coach['userTeam']['teamname']],
                'wins': coach['wins'],
                'draws': coach['draws'],
                'losses': coach['losses'],
                'points_for': coach['points_for'],
                'points_against': coach['points_against'],
                'league_points': coach['points'],
                'position': coach['position']
            }
        )
        df_ladder = pd.concat([df_ladder, df_line])
df_ladder

,round,coach_id,coach_team_id,coach_first_name,coach_team_name,wins,draws,losses,points_for,points_against,league_points,position
0,1,86078,9806,Lester,HoweIMetYourMother,1,0,0,1827,1476,4,1
0,1,180650,33301,Jordan,"What's Up, Doch?",1,0,0,1721,1623,4,2
0,1,45168,5524,Luke,The Dragon's Laird,1,0,0,1720,1662,4,3
0,1,300791,182,Paul,Coronaviney,1,0,0,1686,1657,4,4
0,1,21562,186,Mark,GetRichOrDieTryin,0,0,1,1657,1686,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
0,12,21562,186,Mark,GetRichOrDieTryin,6,0,6,19711,19535,24,4
0,12,45168,5524,Luke,The Dragon's Laird,6,0,6,19781,19846,24,5
0,12,300791,182,Paul,Coronaviney,6,0,6,18999,19318,24,6
0,12,10857,189,Anthony,The Full Bonty,4,0,8,19615,19704,16,7


In [11]:
df_ladder.to_csv('outputs/ladder.csv')

PermissionError: [Errno 13] Permission denied: 'outputs/ladder.csv'

## FIXTURE RESULTS

In [ ]:
df_fixture_results = pd.DataFrame()
for match in match_json_list:
    for result in match['fixtures']:
        df_line = pd.DataFrame(
            {
                'round_number': [result['round']],
                'match_name': '{} vs {}'.format(result['user_team1']['user']['first_name'], result['user_team2']['user']['first_name']),
                'home_team_id': [result['user_team1']['id']],
                'home_team_name': [result['user_team1']['teamname']],
                'home_team_coach': [result['user_team1']['user']['first_name']],
                'home_team_points': [result['user_team1']['stats'][0]['points']],
                'away_team_id': [result['user_team2']['id']],
                'away_team_name': [result['user_team2']['teamname']],
                'away_team_coach': [result['user_team2']['user']['first_name']],
                'away_team_points': [result['user_team2']['stats'][0]['points']],
            }
        )
        df_fixture_results = pd.concat([df_fixture_results, df_line])
    
df_fixture_results

In [ ]:
df_fixture_results.to_csv('outputs/fixture_results.csv'.format(round_number))

### Rearrange fixture data to display by team + opposition scores
- Generate team rank for that week + opponent rank for that week

In [ ]:
df_home_fixture_results = df_fixture_results
df_away_fixture_results = df_fixture_results

home_renamed_cols = {
     'home_team_id': 'team_id',
     'home_team_name': 'team_name',
     'home_team_coach': 'team_coach',
     'home_team_points': 'team_points',
     'away_team_id': 'opposition_team_id',
     'away_team_name': 'opposition_team_name',
     'away_team_coach': 'opposition_team_coach',
     'away_team_points': 'opposition_team_points'
}

away_renamed_cols = {
     'home_team_id': 'opposition_team_id',
     'home_team_name': 'opposition_team_name',
     'home_team_coach': 'opposition_team_coach',
     'home_team_points': 'opposition_team_points',
     'away_team_id': 'team_id',
     'away_team_name': 'team_name',
     'away_team_coach': 'team_coach',
     'away_team_points': 'team_points'
}

df_home_fixture_results = df_home_fixture_results.rename(columns=home_renamed_cols)
df_away_fixture_results = df_away_fixture_results.rename(columns=away_renamed_cols)
df_fixture_by_each_team = pd.concat([df_home_fixture_results, df_away_fixture_results])
df_fixture_by_each_team['Rank'] = df_fixture_by_each_team.groupby('round_number')['team_points'].rank(ascending=False)
df_fixture_by_each_team['Opponent Rank'] = df_fixture_by_each_team.groupby('round_number')['opposition_team_points'].rank(ascending=False)
df_fixture_by_each_team = df_fixture_by_each_team.sort_values(['round_number', 'Rank'])
df_fixture_by_each_team.to_csv('outputs/fixture_results_by_team.csv')
df_fixture_by_each_team

## CURRENTLY LISTED TEAMS

In [ ]:
s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/leagues/96/playersStatus", headers=headers)
players_json = json.loads(s.text)
df_players = pd.DataFrame.from_dict(players_json, orient='index')
df_players = df_players.merge(df_player_list, on='player_id', how='left')
df_currently_listed_players = df_players.loc[df_players['trade_status'] == "owner"]
df_currently_listed_players = df_currently_listed_players.merge(df_coaches, left_on = 'user_team_id', right_on='coach_team_id', how='left')
df_currently_listed_players.to_csv('outputs/current_teams.csv')
df_currently_listed_players

## CURRENT FREE AGENTS + WAIVER

In [ ]:
df_available_players = df_players.loc[df_players['trade_status'] != "owner"]
df_available_players

In [ ]:
df_available_players.to_csv('outputs/current_free_agents_waiver.csv')

# Transactions

## Waiver Picks

In [ ]:
s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/leagues/96/processedWaivers", headers=headers)
waivers_json = json.loads(s.text)
df_waiver_transactions = pd.json_normalize(waivers_json)

wanted_columns = [
    'round',
    'user_team_id',
    'processed',
    'player_id',
    'player.feed_id',
    'player.first_name',
    'player.last_name',
    'player.team.abbrev',
    'dropped_player_id',
    'dropped_player.feed_id',
    'dropped_player.first_name',
    'dropped_player.last_name',
    'dropped_player.team.abbrev',
]

df_waiver_transactions = df_waiver_transactions.loc[:, wanted_columns]
df_waiver_transactions['source'] = "waiver"
df_waiver_transactions

In [ ]:
df_waiver_transactions.to_csv('outputs/waiver_transactions.csv')

## Free Agent trades

In [ ]:
s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/leagues/96/trades", headers=headers)
free_agent_trades_json = json.loads(s.text)
df_fa_transactions = pd.json_normalize(free_agent_trades_json)

wanted_columns = [
    'round',
    'user_id',
    'action_time',
    'buy_player.id',
    'buy_player.feed_id',
    'buy_player.first_name',
    'buy_player.last_name',
    'buy_player.team.abbrev',
    'sell_player.id',
    'sell_player.feed_id',
    'sell_player.first_name',
    'sell_player.last_name',
    'sell_player.team.abbrev',
]

df_fa_transactions = df_fa_transactions.loc[:, wanted_columns]
df_fa_transactions['source'] = "free_agent"
df_fa_transactions

In [ ]:
df_fa_transactions.to_csv('outputs/free_agent_transactions.csv')

## Team Trades

In [ ]:
s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/leagues/96/teamtrades", headers=headers)
team_trades_json = json.loads(s.text)

In [ ]:
df_trades_player_list = pd.DataFrame()

for trade in team_trades_json:
    df_lines = pd.json_normalize(trade['team_trade_players'])
    df_lines['round'] = trade['round']
    df_lines['user_team_id'] = trade['user_team_id']
    df_lines['processed'] = trade['action_time']
    df_lines['status'] = trade['status_id']
    df_lines['source'] = 'trade'
    df_trades_player_list = pd.concat([df_trades_player_list,df_lines])
df_trades_player_list

In [ ]:
df_team_one_trades_player_list = df_trades_player_list.loc[df_trades_player_list['status'] == 7, ['round', 'user_team_id', 'processed', 'player_id', 'source']]
df_team_two_trades_player_list = df_trades_player_list.loc[df_trades_player_list['status'] == 7, ['round', 'user_team_id', 'processed', 'trade_player_id', 'source']]
df_team_two_trades_player_list = df_team_two_trades_player_list.rename(columns={'trade_player_id': 'player_id'})
df_trades_by_player = pd.concat([df_team_one_trades_player_list, df_team_two_trades_player_list]).sort_values(by=['processed'])
df_trades_by_player = df_trades_by_player.merge(df_player_list[['player_id', 'feed_id', 'first_name', 'last_name', 'team']], on='player_id', how='left')
df_trades_by_player = df_trades_by_player.rename(columns={'feed_id': 'player.feed_id', 'first_name': 'player.first_name', 'last_name': 'player.last_name', 'team': 'player.team.abbrev'})
df_trades_by_player

## Combined Free Agents, Waiver, and Trades transactions list

In [ ]:

columns = [
    'round',
    'user_team_id',
    'processed',
    'player_id',
    'player.feed_id',
    'player.first_name',
    'player.last_name',
    'player.team.abbrev',
    'dropped_player_id',
    'dropped_player.feed_id',
    'dropped_player.first_name',
    'dropped_player.last_name',
    'dropped_player.team.abbrev',
    'source',
]
df_waiver_transactions.columns = columns
df_fa_transactions.columns = columns

df_transactions = pd.concat([df_waiver_transactions, df_fa_transactions, df_trades_by_player])
df_transactions = df_transactions.sort_values(by=['processed'])
df_transactions

In [ ]:
df_transactions.to_csv('outputs/transactions.csv')